Import libraries requred for the script

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from numpy.polynomial import polynomial as npp
from scipy.stats import linregress

Read the CSV files

In [2]:
df_pa = pd.read_csv('VMI_Data_BP2_V02_AK6_090118_032619.csv')

Check the columns in the data

In [3]:
df_pa = df_pa.drop('SPLICE_LIMIT_LOW',1)
df_pa = df_pa.drop('SPLICE_LIMIT_HIGH',1)
print(df_pa.columns)


Index(['MACHINE_ID', 'DATE_TIME_STAMP', 'GT_BARCODE', 'TIRE_TYPE',
       'CUT_LENGTH', 'CONV_WAIT_TIME', 'LFT_SPLICE_LENGTH',
       'MID_SPLICE_LENGTH', 'RHT_SPLICE_LENGTH', 'SPLICE_OK', 'BP2_S8_L',
       'BP2_S7_L', 'BP2_S6_L', 'BP2_S5_L', 'BP2_S4_L', 'BP2_S3_L', 'BP2_S2_L',
       'BP2_S1_ML', 'BP2_S1_MR', 'BP2_S2_R', 'BP2_S3_R', 'BP2_S4_R',
       'BP2_S5_R', 'BP2_S6_R', 'BP2_S7_R', 'BP2_S8_R', 'BP2_PART',
       'BP2_BARCODE', 'BP2_BUILD_DATE', 'BP2_MACHINEID'],
      dtype='object')


Sort the data by timestamp

In [4]:
df_pa.DATE_TIME_STAMP = pd.to_datetime(df_pa.DATE_TIME_STAMP, format="%m/%d/%Y %H:%M:%S.%f")


In [5]:
print(df_pa.shape)
df_pa = df_pa.drop_duplicates(subset=['CUT_LENGTH', 'CONV_WAIT_TIME', 'BP2_S8_L', 'BP2_S7_L', 'BP2_S6_L', 'BP2_S5_L', 
                                      'BP2_S4_L', 'BP2_S3_L', 'BP2_S2_L', 'BP2_S1_ML', 'BP2_S1_MR', 'BP2_S2_R', 
                                      'BP2_S3_R', 'BP2_S4_R', 'BP2_S5_R', 'BP2_S6_R', 'BP2_S7_R', 'BP2_S8_R'])
# df_pa = df_pa.drop_duplicates()
print(df_pa.shape)

(138892, 30)
(138052, 30)


Check missing data

In [6]:
df_pa['CUT_LENGTH'] = pd.to_numeric(df_pa['CUT_LENGTH'], errors='coerce')
df_pa['CONV_WAIT_TIME'] = pd.to_numeric(df_pa['CONV_WAIT_TIME'], errors='coerce')
df_pa['LFT_SPLICE_LENGTH'] = pd.to_numeric(df_pa['LFT_SPLICE_LENGTH'], errors='coerce')
df_pa['MID_SPLICE_LENGTH'] = pd.to_numeric(df_pa['MID_SPLICE_LENGTH'], errors='coerce')
df_pa['RHT_SPLICE_LENGTH'] = pd.to_numeric(df_pa['RHT_SPLICE_LENGTH'], errors='coerce')


In [7]:
total = df_pa.isnull().sum().sort_values(ascending=False)
percent = (df_pa.isnull().sum()/df_pa.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(10))
df_pa = df_pa.dropna()



                   Total   Percent
BP2_MACHINEID       5736  4.154956
BP2_BARCODE         5736  4.154956
BP2_PART            5736  4.154956
BP2_BUILD_DATE      5736  4.154956
LFT_SPLICE_LENGTH    126  0.091270
RHT_SPLICE_LENGTH     37  0.026801
MID_SPLICE_LENGTH     18  0.013039
BP2_S6_L               0  0.000000
DATE_TIME_STAMP        0  0.000000
GT_BARCODE             0  0.000000


In [8]:
total = df_pa.isnull().sum().sort_values(ascending=False)
percent = (df_pa.isnull().sum()/df_pa.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(20))



                   Total  Percent
BP2_MACHINEID          0      0.0
BP2_BUILD_DATE         0      0.0
DATE_TIME_STAMP        0      0.0
GT_BARCODE             0      0.0
TIRE_TYPE              0      0.0
CUT_LENGTH             0      0.0
CONV_WAIT_TIME         0      0.0
LFT_SPLICE_LENGTH      0      0.0
MID_SPLICE_LENGTH      0      0.0
RHT_SPLICE_LENGTH      0      0.0
SPLICE_OK              0      0.0
BP2_S8_L               0      0.0
BP2_S7_L               0      0.0
BP2_S6_L               0      0.0
BP2_S5_L               0      0.0
BP2_S4_L               0      0.0
BP2_S3_L               0      0.0
BP2_S2_L               0      0.0
BP2_S1_ML              0      0.0
BP2_S1_MR              0      0.0


In [9]:
df_pa = df_pa.sort_values('DATE_TIME_STAMP')
df_pa.reset_index(drop=True)

,MACHINE_ID,DATE_TIME_STAMP,GT_BARCODE,TIRE_TYPE,CUT_LENGTH,CONV_WAIT_TIME,LFT_SPLICE_LENGTH,MID_SPLICE_LENGTH,RHT_SPLICE_LENGTH,SPLICE_OK,...,BP2_S3_R,BP2_S4_R,BP2_S5_R,BP2_S6_R,BP2_S7_R,BP2_S8_R,BP2_PART,BP2_BARCODE,BP2_BUILD_DATE,BP2_MACHINEID
0,V02,2018-09-01 00:05:28.500,AK6T27O,AK6,1712.241,47.674,5.653774,4.681854,3.664844,0,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
1,V02,2018-09-01 00:10:44.000,AK6T27P,AK6,1712.284,104.228,5.131887,3.666008,2.298164,0,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
2,V02,2018-09-01 00:13:30.900,AK6T27Q,AK6,1712.559,314.116,8.688805,5.854500,5.022215,1,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
3,V02,2018-09-01 00:14:31.700,AK6T27R,AK6,1712.106,138.822,7.528668,6.911073,6.480822,1,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
4,V02,2018-09-01 00:16:52.600,AK6T27S,AK6,1712.358,143.280,7.641249,4.757781,2.509653,0,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
5,V02,2018-09-01 00:17:54.500,AK6T27T,AK6,1712.585,41.690,7.436451,5.801555,6.178861,1,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
6,V02,2018-09-01 00:18:54.200,AK6T27U,AK6,1713.237,113.385,8.167790,6.025553,4.571600,1,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
7,V02,2018-09-01 00:19:52.600,AK6T27V,AK6,1712.383,38.147,6.609984,5.440539,5.547884,1,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
8,V02,2018-09-01 00:28:40.500,AK6T27W,AK6,1711.682,36.172,3.558081,2.276637,0.000000,0,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05
9,V02,2018-09-01 00:29:40.400,AK6T27X,AK6,1712.153,54.665,7.867575,6.172170,5.032106,1,...,0.10,0.10,0.0,0.0,0.0,0.00,VBUCR3E,082818221708VBUCR3E 08,17:08.0,123CTBPH05


Adding ID column for slope calculations
Probably do not need it 

In [10]:

def polyfit(x):
    return npp.polyfit(list(range(len(x))), x, 1)[0]

def linearregress(x):
    slope, intercept, r_value, p_value, std_err = linregress(list(range(len(x))), x)
    return slope


In [11]:
# This code can help write logic for roll length - it uses global var in function

# import pandas as pd
# import numpy as np

# data = np.array([[10, 2, 10, 10],
#                  [10, 3, 60, 100],
#                  [np.nan] * 4,
#                  [10, 22, 280, 250]]).T
# idx = pd.date_range('20150131', end='20150203')
# df = pd.DataFrame(data=data, columns=list('ABCD'), index=idx)
# df
#                A    B     C    D
#  =================================
#  2015-01-31    10   10    NaN  10
#  2015-02-01    2    3     NaN  22 
#  2015-02-02    10   60    NaN  280
#  2015-02-03    10   100   NaN  250

# def calculate(mul, add):
#     global value
#     value = value * mul + add
#     return value

# value = df.loc['2015-01-31', 'D']
# df.loc['2015-01-31', 'C'] = value
# df.loc['2015-02-01':, 'C'] = df.loc['2015-02-01':].apply(lambda row: calculate(*row[['A', 'B']]), axis=1)
# df
#                A    B     C     D
#  =================================
#  2015-01-31    10   10    10    10
#  2015-02-01    2    3     23    22 
#  2015-02-02    10   60    290   280
#  2015-02-03    10   100   3000  250

In [12]:
df_pa['LFT_SPLICE_DELTA'] = df_pa['LFT_SPLICE_LENGTH'] - (3.5+11.0)/2
df_pa['LFT_SPLICE_PREV'] = df_pa.LFT_SPLICE_DELTA.shift(1)
df_pa['LFT_SPLICE_MA5'] = df_pa.LFT_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
df_pa['LFT_SPLICE_MA10'] = df_pa.LFT_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
df_pa['LFT_SPLICE_MA20'] = df_pa.LFT_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
df_pa['LFT_SPLICE_MA50'] = df_pa.LFT_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
df_pa['LFT_SPLICE_SLOPE5'] = df_pa.LFT_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
df_pa['LFT_SPLICE_SLOPE10'] = df_pa.LFT_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
df_pa['LFT_SPLICE_SLOPE20'] = df_pa.LFT_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
df_pa['LFT_SPLICE_SLOPE50'] = df_pa.LFT_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)


df_pa['MID_SPLICE_DELTA'] = df_pa['MID_SPLICE_LENGTH'] - (4.9+18.0)/2
df_pa['MID_SPLICE_PREV'] = df_pa.MID_SPLICE_DELTA.shift(1)
df_pa['MID_SPLICE_MA5'] = df_pa.MID_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
df_pa['MID_SPLICE_MA10'] = df_pa.MID_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
df_pa['MID_SPLICE_MA20'] = df_pa.MID_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
df_pa['MID_SPLICE_MA50'] = df_pa.MID_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
df_pa['MID_SPLICE_SLOPE5'] = df_pa.MID_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
df_pa['MID_SPLICE_SLOPE10'] = df_pa.MID_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
df_pa['MID_SPLICE_SLOPE20'] = df_pa.MID_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
df_pa['MID_SPLICE_SLOPE50'] = df_pa.MID_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)


df_pa['RHT_SPLICE_DELTA'] = df_pa['RHT_SPLICE_LENGTH'] - (3.5+11.0)/2
df_pa['RHT_SPLICE_PREV'] = df_pa.RHT_SPLICE_DELTA.shift(1)
df_pa['RHT_SPLICE_MA5'] = df_pa.RHT_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
df_pa['RHT_SPLICE_MA10'] = df_pa.RHT_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
df_pa['RHT_SPLICE_MA20'] = df_pa.RHT_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
df_pa['RHT_SPLICE_MA50'] = df_pa.RHT_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
df_pa['RHT_SPLICE_SLOPE5'] = df_pa.RHT_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
df_pa['RHT_SPLICE_SLOPE10'] = df_pa.RHT_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
df_pa['RHT_SPLICE_SLOPE20'] = df_pa.RHT_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
df_pa['RHT_SPLICE_SLOPE50'] = df_pa.RHT_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)



In [13]:
# Delete first 50 rows - so that SMA and Slopes are consistent
df_pa = df_pa.iloc[50:]


In [14]:
# lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, (3.5 + df_pa['LFT_SPLICE_LENGTH'].mean())/2,
#             df_pa['LFT_SPLICE_LENGTH'].mean(),(11.0 + df_pa['LFT_SPLICE_LENGTH'].mean())/2,
#             11.0,df_pa['LFT_SPLICE_LENGTH'].max()]

# mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, (4.9 + df_pa['MID_SPLICE_LENGTH'].mean())/2,
#             df_pa['MID_SPLICE_LENGTH'].mean(), (18.0 + df_pa['MID_SPLICE_LENGTH'].mean())/2,
#             18.0,df_pa['MID_SPLICE_LENGTH'].max()]

# rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5,(3.5 + df_pa['RHT_SPLICE_LENGTH'].mean())/2,
#             df_pa['RHT_SPLICE_LENGTH'].mean(),(11.0 + df_pa['RHT_SPLICE_LENGTH'].mean())/2,
#             11.0,df_pa['RHT_SPLICE_LENGTH'].max()]

# lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, (3.5 + (11.0 - 3.5)/3),
#             (11.0 - (11.0 - 3.5)/3), 11.0, df_pa['LFT_SPLICE_LENGTH'].max()]

# mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, (4.9 + (18.0 - 4.9)/3),
#             (18.0 - (18.0 - 4.9)/3), 18.0, df_pa['MID_SPLICE_LENGTH'].max()]

# rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5, (3.5 + (11.0 - 3.5)/3),
#              (11.0 - (11.0 - 3.5)/3), 11.0, df_pa['RHT_SPLICE_LENGTH'].max()]

lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, 11.0, df_pa['LFT_SPLICE_LENGTH'].max()]

mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, 18.0, df_pa['MID_SPLICE_LENGTH'].max()]

rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5, 11.0, df_pa['RHT_SPLICE_LENGTH'].max()]

In [15]:
# bin_names=['Bad','OK','Good','OKH','BadH']
bin_names=['0','1','0H']

In [16]:
# def calSpliceGrade(row):
#     if row['LFT_SPLICE_GRADE'] == 'Bad' or row['MID_SPLICE_GRADE'] == 'Bad' or row['RHT_SPLICE_GRADE'] == 'Bad':
#         return 'Bad'
#     if row['LFT_SPLICE_GRADE'] == 'OK' or row['MID_SPLICE_GRADE'] == 'OK' or row['RHT_SPLICE_GRADE'] == 'OK':
#         return 'OK'
#     return 'Good'

def calSpliceGrade(row):
    if (row['LFT_SPLICE_GRADE'] == 0) or (row['MID_SPLICE_GRADE'] == 0) or (row['RHT_SPLICE_GRADE'] == 0):
        return 0
    return 1

In [17]:
# df['elderly'] = np.where(df['age']>=50, 'yes', 'no')

df_pa['LFT_SPLICE_GRADE'] = pd.cut(df_pa['LFT_SPLICE_LENGTH'],lft_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['MID_SPLICE_GRADE'] = pd.cut(df_pa['MID_SPLICE_LENGTH'],mid_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['RHT_SPLICE_GRADE'] = pd.cut(df_pa['RHT_SPLICE_LENGTH'],rht_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['SPLICE_GRADE'] = df_pa.apply(calSpliceGrade, axis=1) 


In [18]:
#df_pa['LFT_SPLICE_GRADE']
# one_hot=pd.get_dummies(df_pa['LFT_SPLICE_GRADE'])
# df_pa = df_pa.drop('LFT_SPLICE_GRADE', axis = 1)
# df_pa = df_pa.join(one_hot)


In [19]:
#df_pa.head
df_pa.to_csv('VMI_Data_BP2_V02_AK6_090118_032619_mod01.csv', header=True, index=False)
